In [8]:
!pip install Unidecode --quiet
import unidecode 

In [9]:
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords

# Se cargan los corpus


In [10]:
# Montamos el drive
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
with open('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ADHD/corpus_adhd.txt', 'r') as f:
    corpus_adhd = f.read()

In [12]:
# Visualizamos el contenido del corpus de adhd
print(corpus_adhd[:92])

Has anyone tried the GeneSight genetic test for meds? What was your experience? Hi everyone.


## Separamos por documentos

In [13]:
def separate_documents(corpus):
    # Inicialización de variables
    documents = []
    documents = corpus.strip().split('\n\n')

    return documents

In [14]:
doc = separate_documents(corpus_adhd)

In [15]:
doc[3]

'Anyway, my spouse ran at some point into an article about some company called GeneSight that offers genetic tests that supposedly indicate what drugs someone could be receptive or contraindicated for , and one of their tests is for ADHD - they look for certain markers supposedly associated with good or bad dosing responses with various drugs.'

# Creación del script del dataset

In [16]:
# Se crea un script de dataset utilizando la librería de Hugging Face
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.9 MB/s eta 0:00:00


In [17]:
from datasets import Dataset, DatasetDict

In [18]:
# Se crea una lista de diccionarios con los campos "id" y "text" para cada documento en doc
data = [{"id": str(i), "text": doc[i]} for i in range(len(doc))]

In [19]:
# Se crea el dataset con la lista de diccionarios, hacemos el split
train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))
test_size = len(data) - train_size - val_size

train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[train_size+val_size:]

dataset = DatasetDict({'train': train_data, 'validation': val_data, 'test': test_data})

In [20]:
# Obtener los conjuntos de entrenamiento del objeto dataset
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

In [21]:
import os

In [22]:
# Rutas completas donde se almacenarán los archivos de texto de los conjuntos
train_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ADHD/split', 'train.txt')
val_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ADHD/split', 'validation.txt')
test_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ADHD/split', 'test.txt')

In [23]:
# Abrir el archivo de texto para escribir los datos de entrenamiento
with open(train_file_path, 'w', encoding='utf-8') as f:
    for example in train_data:
        # Escribir cada ejemplo de entrenamiento en una línea separada
        f.write(example['text'] + '\n')

# Abrir el archivo de texto para escribir los datos de validacion
with open(val_file_path, 'w', encoding='utf-8') as f:
    for example in val_data:
        # Escribir cada ejemplo de validacion en una línea separada
        f.write(example['text'] + '\n')

# Abrir el archivo de texto para escribir los datos de test
with open(test_file_path, 'w', encoding='utf-8') as f:
    for example in test_data:
        # Escribir cada ejemplo de test en una línea separada
        f.write(example['text'] + '\n')

#Entrenamiento del modelo

In [24]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.2 MB/s eta 0:00:00


In [25]:
!git clone https://github.com/huggingface/transformers.git
!cp transformers/examples/pytorch/language-modeling/run_clm.py /content/transformers/examples/pytorch/language-modeling/

Cloning into 'transformers'...
remote: Enumerating objects: 141843, done.
remote: Counting objects: 100% (1876/1876), done.
remote: Compressing objects: 100% (876/876), done.
remote: Total 141843 (delta 1172), reused 1368 (delta 838), pack-reused 139967
Receiving objects: 100% (141843/141843), 139.91 MiB | 25.07 MiB/s, done.
Resolving deltas: 100% (106145/106145), done.
cp: 'transformers/examples/pytorch/language-modeling/run_clm.py' and '/content/transformers/examples/pytorch/language-modeling/run_clm.py' are the same file


In [26]:
# Configuración del entrenamiento
model_checkpoint = "gpt2" # Modelo base
output_dir = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD" # Carpeta donde se guardarán los modelos y los logs de entrenamiento
num_train_epochs = 3 # Número de épocas de entrenamiento
learning_rate = 5e-5 # Tasa de aprendizaje
per_device_train_batch_size = 2 # Tamaño del lote de entrenamiento por dispositivo
per_device_eval_batch_size = 2 # Tamaño del lote de evaluación por dispositivo
warmup_steps = 500 # Número de pasos de calentamiento
logging_dir = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/logs/generative/ADHD" # Carpeta donde se guardarán los registros de entrenamiento
logging_steps = 500 # Frecuencia de los registros de entrenamiento
save_steps = 500 # Frecuencia de guardar el modelo

In [27]:
cmd = f"python /content/transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path {model_checkpoint} \
    --train_file {train_file_path} \
    --validation_file {val_file_path} \
    --do_train \
    --do_eval \
    --output_dir {output_dir} \
    --num_train_epochs {num_train_epochs} \
    --learning_rate {learning_rate} \
    --per_device_train_batch_size {per_device_train_batch_size} \
    --per_device_eval_batch_size {per_device_eval_batch_size} \
    --warmup_steps {warmup_steps} \
    --logging_dir {logging_dir} \
    --logging_steps {logging_steps} \
    --save_steps {save_steps} \
    --fp16 \
    --overwrite_output_dir \
    --report_to tensorboard"

In [28]:
!pip install -r /content/transformers/examples/pytorch/language-modeling/requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.1 MB/s eta 0:00:00


In [29]:
!pip install git+https://github.com/huggingface/transformers --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [30]:
# Se instala el módulo "evaluate" que no está instalado en el entorno en el que se está ejecutando el script
!pip install evaluate --quiet

In [31]:
%run /content/transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path gpt2 \
    --train_file "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ADHD/split/train.txt" \
    --validation_file "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/ADHD/split/validation.txt" \
    --do_train \
    --do_eval \
    --output_dir "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD" \
    --num_train_epochs 3 \
    --learning_rate 5e-5 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --warmup_steps 500 \
    --logging_dir "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/logs/generative/ADHD" \
    --logging_steps 500 \
    --save_steps 500 \
    --fp16 \
    --overwrite_output_dir \
    --report_to tensorboard

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=

INFO:datasets.download.download_manager:Downloading took 0.0 min
INFO:datasets.download.download_manager:Checksum Computation took 0.0 min


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:datasets.builder:Generating train split


Generating train split: 0 examples [00:00, ? examples/s]

...

[INFO|trainer.py:2904] 2023-05-18 13:58:44,977 >> Saving model checkpoint to /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD/checkpoint-500
[INFO|configuration_utils.py:458] 2023-05-18 13:58:45,671 >> Configuration saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD/checkpoint-500/config.json
[INFO|configuration_utils.py:364] 2023-05-18 13:58:45,985 >> Configuration saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD/checkpoint-500/generation_config.json
[INFO|modeling_utils.py:1836] 2023-05-18 13:58:55,485 >> Model weights saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD/checkpoint-500/pytorch_model.bin
[INFO|tokenization_utils_base.py:2181] 2023-05-18 13:58:55,828 >> tokenizer config file saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD/check

***** train metrics *****
  epoch                    =        3.0
  total_flos               =  3626846GF
  train_loss               =      3.394
  train_runtime            = 0:28:22.70
  train_samples            =       2484
  train_samples_per_second =      4.377
  train_steps_per_second   =      2.188


INFO:__main__:*** Evaluate ***
[INFO|trainer.py:3165] 2023-05-18 14:24:12,621 >> ***** Running Evaluation *****
[INFO|trainer.py:3167] 2023-05-18 14:24:12,622 >>   Num examples = 356
[INFO|trainer.py:3170] 2023-05-18 14:24:12,623 >>   Batch size = 2


***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.3424
  eval_loss               =     3.3226
  eval_runtime            = 0:00:25.20
  eval_samples            =        356
  eval_samples_per_second =     14.125
  eval_steps_per_second   =      7.062
  perplexity              =    27.7333


[INFO|modelcard.py:451] 2023-05-18 14:24:38,583 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.3423945874108977}]}


# Generamos una muestra de texto a partir de una semilla

In [32]:
from transformers import pipeline

In [33]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [34]:
!pip install xformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 9.8 MB/s eta 0:00:00


In [35]:
model_path = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD/checkpoint-3500"

In [36]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

[INFO|tokenization_utils_base.py:1808] 2023-05-18 14:25:14,771 >> loading file vocab.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 14:25:14,773 >> loading file merges.txt
[INFO|tokenization_utils_base.py:1808] 2023-05-18 14:25:14,775 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 14:25:14,778 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 14:25:14,779 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 14:25:14,780 >> loading file tokenizer_config.json
[INFO|configuration_utils.py:667] 2023-05-18 14:25:14,879 >> loading configuration file /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD/checkpoint-3500/config.json
[INFO|configuration_utils.py:725] 2023-05-18 14:25:14,883 >> Model config GPT2Config {
  "_name_or_path": "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/ADHD/checkpoint-3500",


In [37]:
# Definir la configuración del generador de texto
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [38]:
# Definir la semilla para la generación de texto
prompt = "Hello, how are you doing?"

In [39]:
# Generar texto a partir de la semilla
text = generator(prompt, max_length=150, do_sample=True, temperature=0.7)
print(text[0]['generated_text'])

[INFO|configuration_utils.py:577] 2023-05-18 14:25:35,133 >> Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.30.0.dev0"
}

[WARNING|utils.py:1282] 2023-05-18 14:25:35,139 >> Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you doing? I mean, Ive done some work on it and Ive been doing it for like a year and a half now, and I feel a lot better, and I feel like I can focus and do things properly. 
But I feel like Ill never feel like Im doing anything productive. Like Ill forget stuff, I cant remember certain things, i feel like Im falling asleep, Im just trying to do my work.
How do you guys manage this?
TLDR I feel like Im doing nothing productive. 
I can do shit.
I am a kid with ADHD. I have a lot of work to do but I never start anything. When I do start I usually start with chores,
